### Import thư viện

In [24]:
import numpy as np
import tensorflow as tf
import re
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

### Load dữ liệu

In [25]:
with open("./data2.txt", "r", encoding="utf-8") as file:
    data = file.read()

### Xử lý dữ liệu

In [26]:
# Tách từ và loại bỏ ký tự đặc biệt

def word_separation(input_data):
    data = input_data.split('\n')
    token_underthesea = []
    sentences = []

    for sentence in data:
        text = re.sub(r'[^a-zA-Z0-9\sđĐáÁàÀảẢãÃạẠăĂắẮằẰẳẲẵẴạẠâÂấẤầẦẩẨẫẪậẬêÊếẾềỀểỂễỄệỆôÔốỐồỒổỔỗỖộỘơƠớỚờỜởỞỡỠợỢưƯứỨừỪửỬữỮựỰơƠáÁàÀảẢãÃạẠéÉèÈẻẺẽẼếẾềỀểỂễỄếẾêÊấẤầẦẩẨẫẪậẬíÍìÌỉỈĩĨịỊóÓòÒỏỎõÕọỌốỐồỒổỔỗỖộỘơƠớỚờỜởỞỡỠợỢúÚùÙủỦũŨụỤưỪỨừỪửỬữỮựỰýÝỳỲỷỶỹỸỵỴ\s]+', '', sentence).lower()
        sentences.append(text)

        # Tách từ bằng Underthesea
        tokens_underthesea = text.split(' ')
        token_underthesea.append(tokens_underthesea)

    return token_underthesea, sentences

token_underthesea, train_input = word_separation(data)

In [27]:
train_input[:3]

['báo cáo tự đánh giá tđg là một phần của quá trình đảm bảo chất lượng đào tạo tại khoa x',
 'báo cáo này bao gôm 03 phần chính',
 'khái quát nhằm mô tả vắn tắt về báo cáo tđg quá trình tđg và giới thiệu vắn tắt về khoa x']

In [28]:
# Tìm câu có số từ nhiều nhất

word_counts = [len(sentence.split()) for sentence in train_input]

max_word_count = max(word_counts)
index_of_longest_sentence = word_counts.index(max_word_count)
longest_sentence = train_input[index_of_longest_sentence]

print("Số từ trong câu: {}".format(max_word_count))

Số từ trong câu: 258


In [29]:
# Xây dựng bộ từ điển từ danh sách token

def build_vocab(token_underthesea):
    vocab = {}
    for token in token_underthesea:
        for word in token:
            if word not in vocab:
                vocab[word] = len(vocab) + 1
    return vocab


vocab = build_vocab(token_underthesea)
print(vocab)
print("Số lượng từ trong bộ từ điển:", len(vocab))

{'báo': 1, 'cáo': 2, 'tự': 3, 'đánh': 4, 'giá': 5, 'tđg': 6, 'là': 7, 'một': 8, 'phần': 9, 'của': 10, 'quá': 11, 'trình': 12, 'đảm': 13, 'bảo': 14, 'chất': 15, 'lượng': 16, 'đào': 17, 'tạo': 18, 'tại': 19, 'khoa': 20, 'x': 21, 'này': 22, 'bao': 23, 'gôm': 24, '03': 25, 'chính': 26, 'khái': 27, 'quát': 28, 'nhằm': 29, 'mô': 30, 'tả': 31, 'vắn': 32, 'tắt': 33, 'về': 34, 'và': 35, 'giới': 36, 'thiệu': 37, 'theo': 38, 'tiêu': 39, 'chuẩn': 40, 'chí': 41, 'đối': 42, 'với': 43, 'chương': 44, 'ngành': 45, 'bộ': 46, 'ctđt': 47, 'giáo': 48, 'dục': 49, 'ban': 50, 'hành': 51, 'thông': 52, 'tư': 53, '042016ttgdđt': 54, 'ngày': 55, '1432016': 56, 'kết': 57, 'luận': 58, 'tóm': 59, 'những': 60, 'điểm': 61, 'mạnh': 62, 'tồn': 63, 'vấn': 64, 'đề': 65, 'cần': 66, 'cải': 67, 'tiến': 68, 'kế': 69, 'hoạch': 70, 'tổng': 71, 'hợp': 72, 'quả': 73, 'cuối': 74, 'bản': 75, 'phụ': 76, 'lục': 77, 'gồm': 78, 'các': 79, 'tài': 80, 'liệu': 81, 'như': 82, 'quyết': 83, 'định': 84, 'thành': 85, 'lập': 86, 'hội': 87, 'đồn

### Chuẩn bị dữ liệu đầu vào cho mô hình

In [30]:
# Tạo batch size và dataset

dataset = tf.data.Dataset.from_tensor_slices(train_input)

batch_size = 32
batched_dataset = dataset.batch(batch_size)

In [31]:
# Vector hóa dữ liệu bằng TextVectorization layer

sequence_length = max_word_count            # Số lượng từ tối đa trong 1 câu
vocab_size = 50                           # Lớp TextVectorization chỉ xét 10 từ thường gặp nhất.

text_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length,
)
text_vectorization.adapt(batched_dataset)

In [32]:
def prepare_lm_dataset(text_batch):
    vectorized_sequences = text_vectorization(text_batch)
    x = vectorized_sequences[:, :-1]
    y = vectorized_sequences[:, 1:]
    return x, y

lm_dataset = batched_dataset.map(prepare_lm_dataset, num_parallel_calls = 4)    # num_parallel_calls cho phép train song song trên nhiều phần tử

In [33]:
# Chia dữ liệu

val_size = 0.3
val_steps = int(val_size * lm_dataset.cardinality().numpy())
train_steps = lm_dataset.cardinality().numpy() - val_steps

val_dataset = lm_dataset.take(val_steps)
train_dataset = lm_dataset.skip(val_steps)

In [34]:
train_dataset

<_SkipDataset element_spec=(TensorSpec(shape=(None, 257), dtype=tf.int64, name=None), TensorSpec(shape=(None, 257), dtype=tf.int64, name=None))>

### Khởi tạo model transformer và seq2seq

In [35]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dropout(0.2),
             layers.Dense(embed_dim)]
        )

        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


In [36]:
# Tạo mô hình

embed_dim = 256
latent_dim = 1024
num_heads = 2

# Đầu vào là 1 tenxo có dạng (batch_size, sequence_length)
inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [37]:
# Khởi tạo callback để lưu lại trọng số tốt nhất
checkpoint = ModelCheckpoint("./src/best_model_v1.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Train model
model.compile(optimizer=Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history=model.fit(lm_dataset, epochs=20, validation_data=lm_dataset, callbacks=[checkpoint])

Epoch 1/20
104/104 [==============================] - ETA: 0s - loss: 1.9130 - accuracy: 0.6195
Epoch 1: val_accuracy improved from -inf to 0.56533, saving model to ./src\best_model_v1.hdf5
104/104 [==============================] - 131s 1s/step - loss: 1.9130 - accuracy: 0.6195 - val_loss: 1.9553 - val_accuracy: 0.5653
Epoch 2/20


c:\Users\ductr\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


104/104 [==============================] - ETA: 0s - loss: 1.7770 - accuracy: 0.6305
Epoch 2: val_accuracy improved from 0.56533 to 0.62068, saving model to ./src\best_model_v1.hdf5
104/104 [==============================] - 132s 1s/step - loss: 1.7770 - accuracy: 0.6305 - val_loss: 1.7774 - val_accuracy: 0.6207
Epoch 3/20
104/104 [==============================] - ETA: 0s - loss: 1.7308 - accuracy: 0.6348
Epoch 3: val_accuracy improved from 0.62068 to 0.63851, saving model to ./src\best_model_v1.hdf5
104/104 [==============================] - 130s 1s/step - loss: 1.7308 - accuracy: 0.6348 - val_loss: 1.7094 - val_accuracy: 0.6385
Epoch 4/20
104/104 [==============================] - ETA: 0s - loss: 1.6974 - accuracy: 0.6371
Epoch 4: val_accuracy did not improve from 0.63851
104/104 [==============================] - 136s 1s/step - loss: 1.6974 - accuracy: 0.6371 - val_loss: 1.6940 - val_accuracy: 0.6342
Epoch 5/20
104/104 [==============================] - ETA: 0s - loss: 1.6706 - acc

### Dự đoán

In [38]:
import h5py
from keras.models import load_model
import numpy as np

# Định nghĩa custom objects cho việc load model
custom_objects = {'PositionalEmbedding': PositionalEmbedding, 'TransformerDecoder': TransformerDecoder}

# Load mô hình từ tệp HDF5
with h5py.File("./src/best_model_v1.hdf5", "r") as file:
    model_from_memory = load_model(file, custom_objects=custom_objects)

In [40]:
# Tạo một từ điển để ánh xạ từ chỉ mục của từ về từ thực tế
tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)

    return np.argmax(probas)

def predict_next_sentence(keyword, max_length, temp=1.0):
    input_sentence = keyword
    generated_sentence = keyword

    for _ in range(max_length):
        tokenized_sentence = text_vectorization([input_sentence])
        predictions = model_from_memory.predict(tokenized_sentence, verbose=0)
        vectorized_prompt = text_vectorization([input_sentence])[0].numpy()
        prompt_length = np.nonzero(vectorized_prompt == 0)[0][0]
        next_token = sample_next(predictions[0, prompt_length - 1, :], temp)
        sampled_token = tokens_index[next_token]

        if sampled_token == "<END>":
            break

        generated_sentence += " " + sampled_token
        input_sentence = generated_sentence

    return generated_sentence

# Sử dụng hàm để dự đoán câu văn dựa trên từ khóa
def detect(keyword):
    for _ in range(3):
        # keyword = "báo cáo"
        predicted_sentence = predict_next_sentence(keyword, max_length=4, temp=0.2)
        print(predicted_sentence)

keyword = "báo cáo"
detect(keyword)

báo cáo [UNK] [UNK] [UNK] [UNK]
báo cáo [UNK] [UNK] [UNK] [UNK]
báo cáo [UNK] [UNK] [UNK] [UNK]
